# Probabilistic Bayesian Neural Networks
https://keras.io/examples/keras_recipes/bayesian_neural_networks/

## Setup

### Ambiente
Creamos un ambiente con la paquetería necesaria

``conda create -n env_tf_bayes``

``conda activate env_tf_bayes``

``pip install tensorflow-probability``

``pip install tensorflow-datasets``

### Bibliotecas

In [14]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import pandas as pd

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

## Create training and evaluation datasets

In [15]:
def get_train_and_test_splits(train_size, batch_size=1):
    # Importar datos
    data = pd.read_csv("./../data/train.csv")

    # Select the required columns
    cols = [
        'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 
        'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 
        'EstimatedSalary', 'Exited'
    ]
    data = data[cols].copy()
    # One-hot encode the 'Geography' column
    data = pd.get_dummies(data, columns=['Geography'], prefix='Geo')

    # Convert 'Gender' to boolean. 
    # Here we assume 'Male' maps to True and 'Female' to False.
    data['Gender'] = data['Gender'].apply(lambda x: True if x == 'Male' else False)

    # Convert other binary columns to boolean
    bool_cols = ['HasCrCard', 'IsActiveMember', 'Exited']
    data[bool_cols] = data[bool_cols].astype(bool)

    # Ensure 'Age' is integer type
    data['Age'] = data['Age'].astype(int)

    # Scale 'Balance' and 'EstimatedSalary' using MinMaxScaler
    scaler = MinMaxScaler()
    data[['Balance', 'EstimatedSalary']] = scaler.fit_transform(data[['Balance', 'EstimatedSalary']])
    
    # VARIABLE OBJETIVO Y VARIABLES INDEPENDIENTES
    features = data.drop(columns=["Exited"])
    labels = data["Exited"]

    # Convertir a tensores de TensorFlow
    features_dict = {col: tf.convert_to_tensor(features[col].values, dtype=tf.float32) for col in features.columns}
    labels_tensor = tf.convert_to_tensor(labels.values, dtype=tf.float32)

    # Crear dataset de TensorFlow
    dataset = tf.data.Dataset.from_tensor_slices((features_dict, labels_tensor))
    dataset = dataset.cache().shuffle(len(data)).prefetch(buffer_size=tf.data.AUTOTUNE)

    # Definir train_size y test_size correctamente
    test_size = len(data) - train_size  # Corregir tamaño del dataset de prueba

    train_dataset = dataset.take(train_size).batch(batch_size)
    test_dataset = dataset.skip(train_size).take(test_size).batch(batch_size)  # Agregar `take(test_size)`

    return train_dataset, test_dataset

In [16]:
data = pd.read_csv("./../data/train.csv")
dataset_size = data.shape[0]
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

## Compile, train, and evaluate the model

In [17]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):
    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
        loss=loss,
        metrics=['accuracy', tf.keras.metrics.Recall()]
    )

    print("Start training the model...")
    model.fit(
        train_dataset,
        epochs=100,
        validation_data=test_dataset,
        verbose=1
    )
    print("Model training finished.")

    print("Evaluating model performance...")
    test_loss, test_accuracy, test_recall = model.evaluate(test_dataset, verbose=1)
    print('Test Accuracy: {:.2f}%'.format(test_accuracy * 100))
    print('Test Recall: {:.2f}%'.format(test_recall * 100))


## Create model inputs

In [18]:
FEATURE_NAMES = [
    'CreditScore', 
    'Gender', 
    'Age', 
    'Tenure', 
    'Balance', 
    'NumOfProducts', 
    'HasCrCard', 
    'IsActiveMember', 
    'EstimatedSalary', 
    'Geo_France', 
    'Geo_Germany', 
    'Geo_Spain'
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs



## Experiment 1: standard neural network
We create a standard deterministic neural network model as a baseline.

In [19]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    x = keras.layers.concatenate(input_values)
    x = layers.BatchNormalization()(x)

    # Equivalent layers to the Sequential model
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(16, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    # Output layer with sigmoid for binary classification
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


Let's split the wine dataset into training and test sets, with 85% and 15% of the examples, respectively.

In [20]:
data = pd.read_csv("./../data/train.csv")
dataset_size = data.shape[0]
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Now let's train the baseline model. We use the MeanSquaredError as the loss function.

In [21]:
num_epochs = 100
bce_loss = keras.losses.BinaryCrossentropy()
baseline_model = create_baseline_model()
run_experiment(baseline_model, bce_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100


2025-04-15 20:25:04.160043: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [165034]
	 [[{{node Placeholder/_11}}]]
2025-04-15 20:25:04.160513: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [165034]
	 [[{{node Placeholder/_3}}]]


548/548 [==============================] - ETA: 0s - loss: 0.5868 - accuracy: 0.7051 - recall_2: 0.1911

2025-04-15 20:25:10.479451: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype float and shape [165034]
	 [[{{node Placeholder/_9}}]]
2025-04-15 20:25:10.481140: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype float and shape [165034]
	 [[{{node Placeholder/_7}}]]


548/548 [==============================] - 8s 11ms/step - loss: 0.5868 - accuracy: 0.7051 - recall_2: 0.1911 - val_loss: 0.4421 - val_accuracy: 0.7876 - val_recall_2: 0.0000e+00
Epoch 2/100
548/548 [==============================] - 7s 12ms/step - loss: 0.4901 - accuracy: 0.7889 - recall_2: 0.0831 - val_loss: 0.4178 - val_accuracy: 0.8058 - val_recall_2: 0.1181
Epoch 3/100
548/548 [==============================] - 7s 11ms/step - loss: 0.4664 - accuracy: 0.7987 - recall_2: 0.1352 - val_loss: 0.3999 - val_accuracy: 0.8307 - val_recall_2: 0.2710
Epoch 4/100
548/548 [==============================] - 7s 12ms/step - loss: 0.4509 - accuracy: 0.8061 - recall_2: 0.1810 - val_loss: 0.3901 - val_accuracy: 0.8380 - val_recall_2: 0.3700
Epoch 5/100
548/548 [==============================] - 7s 13ms/step - loss: 0.4400 - accuracy: 0.8110 - recall_2: 0.2198 - val_loss: 0.3852 - val_accuracy: 0.8443 - val_recall_2: 0.4177
Epoch 6/100
548/548 [==============================] - 7s 11ms/step - loss: 0.

We take a sample from the test set use the model to obtain predictions for them. Note that since the baseline model is deterministic, we get a single a point estimate prediction for each test example, with no information about the uncertainty of the model nor the prediction.

In [22]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

2025-04-15 20:38:30.654043: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype float and shape [165034]
	 [[{{node Placeholder/_4}}]]
2025-04-15 20:38:30.656280: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [165034]
	 [[{{node Placeholder/_12}}]]


Predicted: 0.0 - Actual: 0.0
Predicted: 0.2 - Actual: 0.0
Predicted: 0.2 - Actual: 0.0
Predicted: 0.0 - Actual: 0.0
Predicted: 0.3 - Actual: 0.0
Predicted: 0.1 - Actual: 0.0
Predicted: 0.1 - Actual: 0.0
Predicted: 0.1 - Actual: 0.0
Predicted: 0.0 - Actual: 0.0
Predicted: 0.5 - Actual: 1.0


## Experiment 2: Bayesian neural network (BNN)

The object of the Bayesian approach for modeling neural networks is to capture the epistemic uncertainty, which is uncertainty about the model fitness, due to limited training data.

The idea is that, instead of learning specific weight (and bias) values in the neural network, the Bayesian approach learns weight distributions - from which we can sample to produce an output for a given input - to encode weight uncertainty.

Thus, we need to define prior and the posterior distributions of these weights, and the training process is to learn the parameters of these distributions.

In [23]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model


We use the tfp.layers.DenseVariational layer instead of the standard keras.layers.Dense layer in the neural network model.

In [24]:

def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1, activation="sigmoid")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


The epistemic uncertainty can be reduced as we increase the size of the training data. That is, the more data the BNN model sees, the more it is certain about its estimates for the weights (distribution parameters). Let's test this behaviour by training the BNN model on a small subset of the training set, and then on the full training set, to compare the output variances.

### Train BNN with a small training subset.

In [25]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, bce_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/100


2025-04-15 20:38:33.124248: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [165034]
	 [[{{node Placeholder/_0}}]]
2025-04-15 20:38:33.124904: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype float and shape [165034]
	 [[{{node Placeholder/_12}}]]


165/165 [==============================] - 6s 23ms/step - loss: 0.5343 - accuracy: 0.7864 - recall_3: 0.0000e+00 - val_loss: 0.5353 - val_accuracy: 0.7848 - val_recall_3: 0.0000e+00
Epoch 2/100
165/165 [==============================] - 3s 19ms/step - loss: 0.5349 - accuracy: 0.7858 - recall_3: 0.0013 - val_loss: 0.5226 - val_accuracy: 0.7928 - val_recall_3: 0.0000e+00
Epoch 3/100
165/165 [==============================] - 5s 26ms/step - loss: 0.5211 - accuracy: 0.7927 - recall_3: 0.0000e+00 - val_loss: 0.5254 - val_accuracy: 0.7887 - val_recall_3: 0.0000e+00
Epoch 4/100
165/165 [==============================] - 4s 20ms/step - loss: 0.5245 - accuracy: 0.7890 - recall_3: 0.0000e+00 - val_loss: 0.5225 - val_accuracy: 0.7898 - val_recall_3: 0.0000e+00
Epoch 5/100
165/165 [==============================] - 4s 19ms/step - loss: 0.5257 - accuracy: 0.7894 - recall_3: 0.0000e+00 - val_loss: 0.5232 - val_accuracy: 0.7895 - val_recall_3: 0.0000e+00
Epoch 6/100
165/165 [=========================

Since we have trained a BNN model, the model produces a different output each time we call it with the same input, since each time a new set of weights are sampled from the distributions to construct the network and produce an output. The less certain the mode weights are, the more variability (wider range) we will see in the outputs of the same inputs.

In [26]:

def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 0.19, min: 0.12, max: 0.3, range: 0.18 - Actual: 0.0
Predictions mean: 0.18, min: 0.11, max: 0.27, range: 0.16 - Actual: 0.0
Predictions mean: 0.18, min: 0.12, max: 0.3, range: 0.18 - Actual: 0.0
Predictions mean: 0.16, min: 0.11, max: 0.25, range: 0.13 - Actual: 0.0
Predictions mean: 0.27, min: 0.18, max: 0.38, range: 0.2 - Actual: 0.0
Predictions mean: 0.2, min: 0.13, max: 0.33, range: 0.2 - Actual: 0.0
Predictions mean: 0.17, min: 0.11, max: 0.25, range: 0.14 - Actual: 0.0
Predictions mean: 0.17, min: 0.12, max: 0.27, range: 0.16 - Actual: 0.0
Predictions mean: 0.14, min: 0.11, max: 0.18, range: 0.07 - Actual: 0.0
Predictions mean: 0.27, min: 0.16, max: 0.37, range: 0.21 - Actual: 1.0


### Train BNN with the whole training set.

In [27]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/100
548/548 [==============================] - 9s 13ms/step - loss: 0.4578 - accuracy: 0.2116 - recall_4: 0.9999 - val_loss: 0.4324 - val_accuracy: 0.2103 - val_recall_4: 1.0000
Epoch 2/100
548/548 [==============================] - 6s 10ms/step - loss: 0.3865 - accuracy: 0.2215 - recall_4: 0.9838 - val_loss: 0.3404 - val_accuracy: 0.2411 - val_recall_4: 0.9534
Epoch 3/100
548/548 [==============================] - 6s 11ms/step - loss: 0.3087 - accuracy: 0.3106 - recall_4: 0.8408 - val_loss: 0.2749 - val_accuracy: 0.4361 - val_recall_4: 0.6420
Epoch 4/100
548/548 [==============================] - 6s 11ms/step - loss: 0.2523 - accuracy: 0.5283 - recall_4: 0.4669 - val_loss: 0.2182 - val_accuracy: 0.7129 - val_recall_4: 0.1612
Epoch 5/100
548/548 [==============================] - 6s 11ms/step - loss: 0.2086 - accuracy: 0.7364 - recall_4: 0.0995 - val_loss: 0.1912 - val_accuracy: 0.7877 - val_recall_4: 0.0146
Epoch 6/100
548/548 [=====================

## Experiment 3: probabilistic Bayesian neural network
So far, the output of the standard and the Bayesian NN models that we built is deterministic, that is, produces a point estimate as a prediction for a given example. We can create a probabilistic NN by letting the model output a distribution. In this case, the model captures the aleatoric uncertainty as well, which is due to irreducible noise in the data, or to the stochastic nature of the process generating the data.

In this example, we model the output as a IndependentNormal distribution, with learnable mean and variance parameters. If the task was classification, we would have used IndependentBernoulli with binary classes, and OneHotCategorical with multiple classes, to model distribution of the model output.

In [28]:

def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    ##############################
    #Since we're modeling a binary target, the probabilistic output should be a Bernoulli distribution, not a Normal one.
    logits = layers.Dense(units=1)(features)
    outputs = tfp.layers.DistributionLambda(lambda t: tfp.distributions.Bernoulli(logits=t))(logits)


    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


Since the output of the model is a distribution, rather than a point estimate, we use the negative loglikelihood as our loss function to compute how likely to see the true data (targets) from the estimated distribution produced by the model.

In [29]:
#Keep our negative_loglikelihood() function but use it with the Bernoulli distribution
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(tf.cast(targets, tf.float32))


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
548/548 [==============================] - 9s 12ms/step - loss: 0.6794 - accuracy: 0.5141 - recall_5: 0.4732 - val_loss: 0.6310 - val_accuracy: 0.5404 - val_recall_5: 0.4239
Epoch 2/100
548/548 [==============================] - 7s 11ms/step - loss: 0.5946 - accuracy: 0.5665 - recall_5: 0.3828 - val_loss: 0.5589 - val_accuracy: 0.5979 - val_recall_5: 0.3357
Epoch 3/100
548/548 [==============================] - 7s 11ms/step - loss: 0.5458 - accuracy: 0.6130 - recall_5: 0.3026 - val_loss: 0.5285 - val_accuracy: 0.6346 - val_recall_5: 0.2691
Epoch 4/100
548/548 [==============================] - 5s 9ms/step - loss: 0.5245 - accuracy: 0.6469 - recall_5: 0.2516 - val_loss: 0.5212 - val_accuracy: 0.6509 - val_recall_5: 0.2384
Epoch 5/100
548/548 [==============================] - 5s 9ms/step - loss: 0.5214 - accuracy: 0.6629 - recall_5: 0.2263 - val_loss: 0.5166 - val_accuracy: 0.6669 - val_recall_5: 0.2103
Epoch 6/100
548/548 [=======================

Now let's produce an output from the model given the test examples. The output is now a distribution, and we can use its mean and variance to compute the confidence intervals (CI) of the prediction.

In [30]:
#To get probabilities from the Bernoulli distribution:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().flatten().tolist()

# You won’t have .stddev() or .mean ± 1.96 * stddev like in the Normal case. Instead, you can:
# sample multiple times from the distribution
# report percentiles (like 5%–95%) to express uncertainty



for idx in range(sample):
    prob = prediction_mean[idx]
    print(f"Predicted probability: {round(prob, 2)} - Actual: {targets[idx]}")


Predicted probability: 0.06 - Actual: 0.0
Predicted probability: 0.12 - Actual: 0.0
Predicted probability: 0.15 - Actual: 0.0
Predicted probability: 0.06 - Actual: 0.0
Predicted probability: 0.31 - Actual: 0.0
Predicted probability: 0.24 - Actual: 0.0
Predicted probability: 0.12 - Actual: 0.0
Predicted probability: 0.09 - Actual: 0.0
Predicted probability: 0.05 - Actual: 0.0
Predicted probability: 0.48 - Actual: 1.0


In [31]:
# Sample multiple times from the Bernoulli distributions
samples = []
num_samples = 100

for _ in range(num_samples):
    samples.append(prob_bnn_model(examples).sample().numpy().flatten())

# Shape: (num_samples, batch_size)
samples_array = np.array(samples)

# Compute percentiles
p5 = np.percentile(samples_array, 5, axis=0)
p95 = np.percentile(samples_array, 95, axis=0)
mean_probs = prediction_distribution.mean().numpy().flatten()

for idx in range(sample):
    print(
        f"Mean: {round(mean_probs[idx], 2)} - "
        f"5th percentile: {round(p5[idx], 2)} - "
        f"95th percentile: {round(p95[idx], 2)} - "
        f"Actual: {targets[idx]}"
    )


Mean: 0.05999999865889549 - 5th percentile: 0.0 - 95th percentile: 1.0 - Actual: 0.0
Mean: 0.11999999731779099 - 5th percentile: 0.0 - 95th percentile: 1.0 - Actual: 0.0
Mean: 0.15000000596046448 - 5th percentile: 0.0 - 95th percentile: 1.0 - Actual: 0.0
Mean: 0.05999999865889549 - 5th percentile: 0.0 - 95th percentile: 1.0 - Actual: 0.0
Mean: 0.3100000023841858 - 5th percentile: 0.0 - 95th percentile: 1.0 - Actual: 0.0
Mean: 0.23999999463558197 - 5th percentile: 0.0 - 95th percentile: 1.0 - Actual: 0.0
Mean: 0.11999999731779099 - 5th percentile: 0.0 - 95th percentile: 1.0 - Actual: 0.0
Mean: 0.09000000357627869 - 5th percentile: 0.0 - 95th percentile: 1.0 - Actual: 0.0
Mean: 0.05000000074505806 - 5th percentile: 0.0 - 95th percentile: 0.0 - Actual: 0.0
Mean: 0.47999998927116394 - 5th percentile: 0.0 - 95th percentile: 1.0 - Actual: 1.0
